In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv ../.env.local

In [7]:
import base64
from anthropic import Anthropic

import os
from pathlib import Path
import json

from utils import build_message_list, parse_xml_response

In [3]:
client = Anthropic(
    api_key=os.environ.get('ANTHROPIC_API_KEY'),
)

MODEL_NAME = "claude-3-opus-20240229"

In [12]:
graph = {'nodes': [{'type': 'text',
   'text': "\n{$CONCEPT}\n</Inputs>\n\n<Instructions Structure>\n- Introduction explaining the purpose of the instructions\n- Ask the AI to write down the concept it will be teaching inside <concept> tags\n- Provide step-by-step instructions for the AI to follow to create the tutorial, including:\n  - Researching the concept and gathering key information \n  - Breaking down the concept into smaller, more manageable parts\n  - Explaining each part in detail with examples\n  - Providing practice exercises or questions to reinforce understanding\n  - Summarizing the key takeaways \n- Remind the AI to present the information in a clear, engaging way suitable for beginners\n- Have the AI output the final tutorial inside <tutorial> tags\n</Instructions Structure>\n\n<Instructions>\nYou will be creating a step-by-step tutorial to teach a beginner how to learn a new concept. The goal is to break the concept down into understandable parts, explain each part in detail, and provide examples and practice to reinforce learning.\n\nFirst, please write the concept you will be teaching inside <concept> tags:\n\n<concept>\n{$CONCEPT}\n</concept>\n\nNow, follow these steps to create the tutorial:\n\n1. Research the concept thoroughly using reliable sources. Gather the key facts, definitions, and foundational knowledge needed to understand it. \n\n2. Break the concept down into smaller subtopics or components. Think about how the information can be organized and presented in a logical order.\n\n3. For each subtopic or component, provide a clear explanation aimed at beginners. Define any new terms and illustrate the ideas with examples. As you write the explanations, think about what a beginner would need to know to grasp each part.\n\n4. After explaining each component, include a few practice exercises, questions or examples that will allow the learner to apply and reinforce their understanding. Provide the answers so they can check their work.\n\n5. Once you've explained all the components, write a summary of the key points and takeaways from the tutorial. Reiterate the most important things for the learner to remember.\n\nWhen writing your explanations and examples, aim to make the tutorial engaging and easy to follow for beginners. Use simple language, define terms clearly, and present the information in an organized way with a logical flow. \n\nAfter completing all the steps, please output the final tutorial text inside <tutorial> tags.\n</Instructions>",
   'id': '7d9b1d6e2a1795e3',
   'x': -135,
   'y': -160,
   'width': 270,
   'height': 60,
   'color': '4'},
  {'type': 'text',
   'text': 'User enters the concept that they want to learn',
   'id': 'c05f05c824cc0a17',
   'x': -135,
   'y': -40,
   'width': 270,
   'height': 60},
  {'type': 'text',
   'text': '',
   'id': '6924aba137eda0b8',
   'x': -67,
   'y': 80,
   'width': 135,
   'height': 60,
   'color': '6'}],
 'edges': [{'id': 'a646b5e1e39cc2a1',
   'fromNode': '7d9b1d6e2a1795e3',
   'fromSide': 'bottom',
   'toNode': 'c05f05c824cc0a17',
   'toSide': 'top'},
  {'id': '8c4b02d8b8d5b7fd',
   'fromNode': 'c05f05c824cc0a17',
   'fromSide': 'bottom',
   'toNode': '6924aba137eda0b8',
   'toSide': 'top'}]}

variables_by_node = {}
for node in graph['nodes']:
    if 'text' in node:
        variables = set([v[2:-1] for v in node['text'].split() if v.startswith('{$')])
        if variables:
            variables_by_node[node['id']] = variables_by_node.get(node['id'], set()).union(variables)
        for variable in variables:
            node['text'] = node['text'].replace(f'{{${variable}}}', f'{{{{{variable.lower()}}}}}')

def format_variables(variables_by_node):
    output = ''
    for node, variables in variables_by_node.items():
        for variable in variables:
            output += f'Variable "{variable.lower()}" connects to node "{node}"\n'
    return output

print(format_variables(variables_by_node))
print(json.dumps(graph, indent=2))


Variable "concept" connects to node "7d9b1d6e2a1795e3"

{
  "nodes": [
    {
      "type": "text",
      "text": "\n{{concept}}\n</Inputs>\n\n<Instructions Structure>\n- Introduction explaining the purpose of the instructions\n- Ask the AI to write down the concept it will be teaching inside <concept> tags\n- Provide step-by-step instructions for the AI to follow to create the tutorial, including:\n  - Researching the concept and gathering key information \n  - Breaking down the concept into smaller, more manageable parts\n  - Explaining each part in detail with examples\n  - Providing practice exercises or questions to reinforce understanding\n  - Summarizing the key takeaways \n- Remind the AI to present the information in a clear, engaging way suitable for beginners\n- Have the AI output the final tutorial inside <tutorial> tags\n</Instructions Structure>\n\n<Instructions>\nYou will be creating a step-by-step tutorial to teach a beginner how to learn a new concept. The goal is to br

In [13]:
system_prompt = '''You are a AI engineer working on a prompt workflow. 
You have been tasked with creating a JSON Canvas diagram that shows the flow of prompts from a handwritten sketch.

To add parameters to the JSON Canvas diagram, you need to add a new node and an edge connecting it to the existing node where the parameter is used.
All parameter nodes have color "6" and are represented as empty text nodes.
Make sure that the new parameter node does not overlap with any existing nodes.

For example, the following parameters produced the following JSON Canvas diagram:

<canvas>
{
  "nodes": [
    ...,
    {
      "type": "text",
      "text": "What is the capital of {{country}}",
      "id": "c05f05c824cc0a17",
      "x": -135,
      "y": -160,
      "width": 270,
      "height": 60
    }
  ],
  "edges": [
    ...
  ]
}
</canvas>

Add the following parameters to the JSON Canvas diagram:

<parameters>
Variable "country" connects to node "c05f05c824cc0a17"
</parameters>

<canvas>
{
  "nodes": [
    ...,
    {
      "type": "text",
      "text": "What is the capital of {{country}}?",
      "id": "c05f05c824cc0a17",
      "x": -135,
      "y": -160,
      "width": 270,
      "height": 60
    },
    {
      "type": "text",
      "text": "",
      "id": "7d9b1d6e2a1795e3",
      "x": -67,
      "y": 80,
      "width": 135,
      "height": 60,
      "color": "6"
    }
  ],
  "edges": [
    ...,
    {
      "id": "a646b5e1e39cc2a1",
      "label": "country"
      "fromNode": "7d9b1d6e2a1795e3",
      "fromSide": "bottom",
      "toNode": "c05f05c824cc0a17",
      "toSide": "top"
    }
  ]
}
</canvas>
'''

user_prompt = f'''<canvas>
{json.dumps(graph)}
</canvas>

Add the following parameters to the JSON Canvas diagram:

<parameters>
{format_variables(variables_by_node)}
</parameters>
'''

message_list = build_message_list('../data/metaprompt.png', user_prompt)

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    system=system_prompt,
    messages=message_list,
	temperature=0.0,
)

print(response.content[0].text)

Here is the updated JSON Canvas diagram with the new "concept" parameter node connected to the instructions node:

<canvas>
{"nodes": [{"type": "text", "text": "\n{{concept}}\n</Inputs>\n\n<Instructions Structure>\n- Introduction explaining the purpose of the instructions\n- Ask the AI to write down the concept it will be teaching inside <concept> tags\n- Provide step-by-step instructions for the AI to follow to create the tutorial, including:\n  - Researching the concept and gathering key information \n  - Breaking down the concept into smaller, more manageable parts\n  - Explaining each part in detail with examples\n  - Providing practice exercises or questions to reinforce understanding\n  - Summarizing the key takeaways \n- Remind the AI to present the information in a clear, engaging way suitable for beginners\n- Have the AI output the final tutorial inside <tutorial> tags\n</Instructions Structure>\n\n<Instructions>\nYou will be creating a step-by-step tutorial to teach a beginne

In [14]:
obsidian_vault = Path(os.environ.get('OBSIDIAN_VAULT'))
output_file = obsidian_vault / 'params.canvas'

json_data = parse_xml_response(response)
with open(output_file, 'w') as f:
    f.write(json.dumps(json_data, indent=4))